In [32]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os
from dotenv import load_dotenv
load_dotenv()

True

In [33]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [34]:
#extract pdf data

def load_pdf(data):
    loader = DirectoryLoader(data,glob="*pdf",
                    loader_cls=PyPDFLoader)
    docs = loader.load()
    return docs

In [35]:
extracted_data = load_pdf(r"C:\D\KODING SHITZ\genai\medical_chatbot\data")

In [36]:
#create text chunks

def text_splitter(extracted_data):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
    chunks = splitter.split_documents(extracted_data)
    return chunks

In [37]:
text_chunks = text_splitter(extracted_data)
len(text_chunks)

7486

In [38]:
#download embedding model
def download_hugging_face_embedding():
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

embedding2 = download_hugging_face_embedding()

In [39]:
embedding2

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [40]:
query_result = embedding2.embed_query("Hello")
len(query_result)

384

In [41]:
# from pinecone import Pinecone as PC
# from pinecone import ServerlessSpec
# pc = PC(api_key=PINECONE_API_KEY)

In [42]:
# index_name="medical-chatbot"
# index = pc.Index(index_name)
# index

In [43]:
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings()
len(embedding.embed_query("Hello"))

1536

In [44]:
from langchain_chroma import Chroma
persist_directory="C:\D\KODING SHITZ\genai\medical_chatbot\db"
vectorstore = Chroma.from_documents(
    text_chunks,
    embedding=embedding,
    persist_directory=persist_directory
)

In [45]:
vectorstore.similarity_search("what are allergies",k=3)

[Document(metadata={'page': 128, 'source': 'C:\\D\\KODING SHITZ\\genai\\medical_chatbot\\data\\Medical_book.pdf'}, page_content='Description\nAllergies are among the most common of medical\ndisorders. It is estimated that 60 million Americans, ormore than one in every five people, suffer from someform of allergy, with similar proportions throughoutmuch of the rest of the world. Allergy is the single largestreason for school absence and is a major source of lostproductivity in the workplace.\nAn allergy is a type of immune reaction. Normally,'),
 Document(metadata={'page': 128, 'source': 'C:\\D\\KODING SHITZ\\genai\\medical_chatbot\\data\\Medical_book.pdf'}, page_content='Description\nAllergies are among the most common of medical\ndisorders. It is estimated that 60 million Americans, ormore than one in every five people, suffer from someform of allergy, with similar proportions throughoutmuch of the rest of the world. Allergy is the single largestreason for school absence and is a majo

In [46]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [47]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [48]:
llm=CTransformers(model="C:\D\KODING SHITZ\genai\medical_chatbot\model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [49]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vectorstore.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [50]:
while True:
    user_input = input(f"Input Prompt")
    result = qa({"query":user_input})
    print("Response: ",result["result"])

c:\Users\hp\anaconda3\envs\finalv1\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response:  Tachy
Tachyess
According to be informed response to treatments. Tachy
Tachyepine.
Response:  The purpose of antipsychotic drugs is to treat psychosis and other mental and emotional conditions, such as schizophrenia, characterized by defective or lost contact with reality, often accompanied by hallucinations or delusions.
